In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import requests

In [2]:
chrome_driver = 'C:/chromedriver.exe' #chromedriver_location

s = Service(chrome_driver)
chrome = webdriver.Chrome(service=s)  #initializing chromedriver


directory = 'ATM_ALL_BANKS_RBI'
if not os.path.exists(directory):
    os.makedirs(directory)     #Creating a folder (Directory) to download files

In [3]:
for year in range(2022, 2024):   #looping through the years from 2022 to 2024
    
    url = 'https://www.rbi.org.in/Scripts/ATMView.aspx' #RBI website to get the data
    
    chrome.get(url)  #opening a new chrome window

    wait = WebDriverWait(chrome, 70) #delay for loading the webpage
    
    year_button = wait.until(EC.element_to_be_clickable((By.ID, f'btn{year}')))
    year_button.click()  #opening the year side

    start_month = 4 if year == 2022 else 1  
    end_month = 13 if year == 2022 else 4  #range of months as per the year

    for month in range(start_month, end_month): #looping through months on the webpage
       
        month_link_id = f'{year}{month}'
        month_link = wait.until(EC.presence_of_element_located((By.ID, month_link_id))) #

        chrome.execute_script("arguments[0].click();", month_link) #Clicking the months

        download = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href$=".XLSX"]')))
        download_url = download.get_attribute('href')  #searching the download link using href attribute

        output = requests.get(download_url)
        file_path = os.path.join(directory, f'{month:02d}_{year:0d}.xlsx') #downloading the file
        with open(file_path, 'wb') as file:
            file.write(output.content)